# Chapter 3: Processing Images with OpenCV

This Jupyter Notebook allows you to interactively edit and run a subset of the code samples from the corresponding chapter in our book, *Learning OpenCV 5 Computer Vision with Python 3*.

Any Jupyter server should be capable of running the Notebook, even if the sample input images files are not available in the server's local filesystem. For example, you can run the Notebook in Google Colab by opening the following link in your Web browser: https://colab.research.google.com/github/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition/blob/main/chapter03/chapter03.ipynb. Specifically, this link opens the Notebook's latest version, hosted on GitHub.

For additional code samples and instructions, please refer to the book and to the GitHub repository at https://github.com/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition. Bear in mind that many of the book's code samples involve camera input or video input/output, which is not well suited to the Jupyter server environment, so there is more to explore beyond Jupyter!

## Upgrading OpenCV and running the compatibility script

**IMPORTANT:** Run the scripts in this section first and run them in order; otherwise, code in subsequent sections may fail or hang.

If you are running this Notebook in Google Colab or another environment where OpenCV might not be up-to-date, run the following command to upgrade the OpenCV pip package:

In [ ]:
!pip install opencv-contrib-python --upgrade

If the preceding command's output includes a prompt to restart the kernel, do restart it.

Now, run the following script, which provides a compatibility layer between OpenCV and Jupyter:

In [ ]:
# %load ../compat/jupyter_compat.py
import os

import cv2
import numpy
import PIL.Image

from IPython import display
from urllib.request import urlopen


def cv2_imshow(winname, mat):
    mat = mat.clip(0, 255).astype('uint8')
    if mat.ndim == 3:
        if mat.shape[2] == 4:
            mat = cv2.cvtColor(mat, cv2.COLOR_BGRA2RGBA)
        else:
            mat = cv2.cvtColor(mat, cv2.COLOR_BGR2RGB)
    display.display(PIL.Image.fromarray(mat))

cv2.imshow = cv2_imshow


def cv2_waitKey(delay=0):
    return -1

cv2.waitKey = cv2_waitKey


def cv2_imread(filename, flags=cv2.IMREAD_COLOR):
    if os.path.exists(filename):
        image = cv2._imread(filename, flags)
    else:
        url = f'https://github.com/PacktPublishing/Learning-OpenCV-5-Computer-Vision-with-Python-Fourth-Edition/raw/main/*/{filename}'
        resp = urlopen(url)
        image = numpy.asarray(bytearray(resp.read()), dtype='uint8')
        image = cv2.imdecode(image, flags)
    return image

# Cache the original implementation of `imread`, if we have not already
# done so on a previous run of this cell.
if '_imread' not in dir(cv2):
    cv2._imread = cv2.imread

cv2.imread = cv2_imread


What did we just do? We imported OpenCV and we replaced some of OpenCV's I/O functions with our own functions that do not rely on a windowed environment or on a local filesystem.

## Applying high-pass and low-pass filters

Let's experiment with high-pass filters (HPFs), which can highlight the edges in an image, and low-pass filters (LPFs), which can blur an image.

Run the following script, which applies HPFs and LPFs to a photo of a statue of an angel, with a clear sky in the background:

In [ ]:
# %load hpf.py
import cv2
import numpy as np
from scipy import ndimage

kernel_3x3 = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])

kernel_5x5 = np.array([[-1, -1, -1, -1, -1],
                       [-1,  1,  2,  1, -1],
                       [-1,  2,  4,  2, -1],
                       [-1,  1,  2,  1, -1],
                       [-1, -1, -1, -1, -1]])

img = cv2.imread("../images/statue_small.jpg",
                 cv2.IMREAD_GRAYSCALE)

k3 = ndimage.convolve(img, kernel_3x3)
k5 = ndimage.convolve(img, kernel_5x5)

blurred = cv2.GaussianBlur(img, (17,17), 0)
g_hpf = img - blurred

cv2.imshow("3x3", k3)
cv2.imshow("5x5", k5)
cv2.imshow("blurred", blurred)
cv2.imshow("g_hpf", g_hpf)
cv2.waitKey()
cv2.destroyAllWindows()


You probably see that a lot of noise is highlighted, as well as real edges. However, if we blur the image, the noise is reduced. Experiment with the kernel sizes and values to see how the results are affected.

## Edge detection with Canny

For better edge detection results, we can apply the Canny algorithm to the same image, as follows:



In [ ]:
# %load canny.py
import cv2
import numpy as np

img = cv2.imread("../images/statue_small.jpg", 0)
cv2.imwrite("canny.jpg", cv2.Canny(img, 200, 300))
cv2.imshow("canny", cv2.imread("canny.jpg"))
cv2.waitKey()
cv2.destroyAllWindows()


The `Canny` function's threshold parameters (such as `200, 300`) determine how sensitive the edge detector is. The first threshold affects the filter's first pass where it looks for any edges, and the second threshold affects the filter's second pass where it attempts to find more edges connected to the first ones. Experiment with the parameters to see how they affect the result.

## Bounding box, minimum area rectangle, and minimum enclosing circle

OpenCV is effective at detecting and analyzing contours in high-contrast images, such as illustrations. The following script detects contours in an illustration of Thor's Hammer (a prominent symbol from Norse mythology), and then it analyzes those contours to find a bounding box, minimum area rectangle, and minimum enclosing circle:

In [ ]:
# %load contours_2.py
import cv2
import numpy as np

OPENCV_MAJOR_VERSION = int(cv2.__version__.split('.')[0])

img = cv2.pyrDown(cv2.imread("../images/hammer.jpg"))

ret, thresh = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
                            127, 255, cv2.THRESH_BINARY)

if OPENCV_MAJOR_VERSION >= 4:
    # OpenCV 4 or a later version is being used.
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                      cv2.CHAIN_APPROX_SIMPLE)
else:
    # OpenCV 3 or an earlier version is being used.
    # cv2.findContours has an extra return value.
    # The extra return value is the thresholded image, which is
    # unchanged, so we can ignore it.
    _, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # find bounding box coordinates
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)

    # find minimum area
    rect = cv2.minAreaRect(c)
    # calculate coordinates of the minimum area rectangle
    box = cv2.boxPoints(rect)
    # normalize coordinates to integers
    box = np.int0(box)
    # draw contours
    cv2.drawContours(img, [box], 0, (0, 0, 255), 3)

    # calculate center and radius of minimum enclosing circle
    (x, y), radius = cv2.minEnclosingCircle(c)
    # cast to integers
    center = (int(x), int(y))
    radius = int(radius)
    # draw the circle
    img = cv2.circle(img, center, radius, (0, 255, 0), 2)

cv2.drawContours(img, contours, -1, (255, 0, 0), 1)
cv2.imshow("contours", img)

cv2.waitKey()
cv2.destroyAllWindows()


Next, let's look at how to generate other kinds of geometric shapes that more tightly fit the detected contours.

## Convex contours and the Douglas-Peucker algorithm

The Douglas-Peucker algorithm, as implemented in OpenCV's `approxPolyDP` function, can approximate contours as polygons, with a specified level of precision. Moreover, the `convexHull` function can find a convex shape that best fits the contours. Here is an example where we applies these two functions to the contours of Thor's Hammer:

In [ ]:
# %load contours_hull.py
import cv2
import numpy as np

OPENCV_MAJOR_VERSION = int(cv2.__version__.split('.')[0])

img = cv2.pyrDown(cv2.imread("../images/hammer.jpg"))

ret, thresh = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
                            127, 255, cv2.THRESH_BINARY)

if OPENCV_MAJOR_VERSION >= 4:
    # OpenCV 4 or a later version is being used.
    contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                      cv2.CHAIN_APPROX_SIMPLE)
else:
    # OpenCV 3 or an earlier version is being used.
    # cv2.findContours has an extra return value.
    # The extra return value is the thresholded image, which is
    # unchanged, so we can ignore it.
    _, contours, hier = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)

black = np.zeros_like(img)
for cnt in contours:
    epsilon = 0.01 * cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    hull = cv2.convexHull(cnt)
    cv2.drawContours(black, [cnt], -1, (0, 255, 0), 2)
    cv2.drawContours(black, [approx], -1, (255, 255, 0), 2)
    cv2.drawContours(black, [hull], -1, (0, 0, 255), 2)

cv2.imshow("hull", black)
cv2.waitKey()
cv2.destroyAllWindows()


Try adjusting the `epsilon` parameter to see how it affects the results of `approxPolyDP`.


## Detecting lines

While OpenCV's contour detection works well on illustrations, it is less effective when applied to photographs or other noisy, detailed images. If we need to detect simple geometric shapes in photographs, a more robust option is to use a combination of the Canny and Hough algorithms.

The following code sample uses Canny and Hough to detect lines in a photo of a train platform:

In [ ]:
# %load hough_lines.py
import cv2
import numpy as np

img = cv2.imread('../images/houghlines5.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 120)

lines = cv2.HoughLinesP(edges, rho=1,
                        theta=np.pi/180.0,
                        threshold=20,
                        minLineLength=40,
                        maxLineGap=5)
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2.imshow("edges", edges)
cv2.imshow("lines", img)
cv2.waitKey()
cv2.destroyAllWindows()


Try adjusting `minLineLength`, `maxLineGap`, and other parameters to see how the detection results are affected.

## Detecting circles

Likewise, the Canny and Hough algorithms can be used for circle detection. The `HoughCircles` function actually implements both Canny and Hough together, so we do not need to call the `Canny` function separately. Here is an example, where we detect circles in a stylized image of our solar system's planets:

In [ ]:
# %load hough_circles.py
import cv2
import numpy as np

planets = cv2.imread("../images/planet_glow.jpg")
gray_img = cv2.cvtColor(planets, cv2.COLOR_BGR2GRAY)
gray_img = cv2.medianBlur(gray_img, 5)

circles = cv2.HoughCircles(gray_img, cv2.HOUGH_GRADIENT,
                           1, 120, param1=90, param2=40,
                           minRadius=0, maxRadius=0)

if circles is not None:
    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(planets, (i[0], i[1]), i[2],
                   (0, 255, 0), 2)
        # draw the center of the circle
        cv2.circle(planets, (i[0], i[1]), 2,
                   (0, 0, 255), 3)

cv2.imwrite("planets_circles.jpg", planets)
cv2.imshow("HoughCircles", planets)
cv2.waitKey()
cv2.destroyAllWindows()


Again, try adjusting the parameters to see how the results change.

# Summary

That is all for now! Please refer to the book and to the GitHub repository for additional samples, including integration with an interactive camera application.